In [1]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from pandas import DataFrame
from patsy import dmatrices
from patsy import dmatrix
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
import matplotlib.pyplot as plt
import matplotlib
import sys

In [2]:
from time import time
from datetime import date, timedelta
td = timedelta(days=1)

import os
os.environ['QT_API'] = 'pyside'
%pylab qt


Populating the interactive namespace from numpy and matplotlib


In [ ]:
all_info = pd.read_csv('160205_cluster/integrated_0or1_larger_sect_with_cluster.csv', encoding='utf-8-sig')
mania_chart = pd.read_csv('160130/mania_data_revised.csv', encoding='utf-8-sig')
freq_artists = pd.read_csv('160204_advanced/freqArtists.csv', encoding='utf-8-sig')
freq_songs = pd.read_csv('160204_advanced/freqSongs.csv', encoding='utf-8-sig')

In [140]:
temp = {}
lrdf = DataFrame(temp)

#features = 'meanT'
#features = 'meanT + avgH + avgW'
#features = 'meanT + avgH + avgW + Season + Rain'
#features = 'meanT + avgH + avgW + Rain + Month'
#features = 'T0 + T1 + T2 + T3 + T4 + T5 + T6 + T7 + T8 + T9 + H0 + H1 + H2 + H3 + H4 + H5 + H6 + H7 + H8 + H9 + W0 + W1 + Rain + Snow + Month + heat_index + discomfort_index'
#features = 'T0 + T1 + T2 + T3 + T4 + T5 + T6 + T7 + T8 + T9'
#features = 'H0 + H1 + H2 + H3 + H4 + H5 + H6 + H7 + H8 + H9'
#features = 'W0 + W1'
# features = 'T0 + T1 + T2 + T3 + T4 + T5 + T6 + T7 + T8 + T9 + H0 + H1 + H2 + H3 + H4 + H5 + H6 + H7 + H8 + H9 + W0 + W1\
#             + season_spring + season_summer + season_fall + season_winter + Rain + Snow + Fog + month_1 + month_2 + month_3 + month_4 + month_5\
#             + month_6 + month_7 + month_8 + month_9 + month_10 + month_11 + month_12'
features = 'T0 + T1 + T2 + T3 + T4 + T5 + T6 + T7 + T8 + T9 + H0 + H1 + H2 + H3 + H4 + H5 + H6 + H7 + H8 + H9 + W0 + W1\
            + season_spring + season_summer + season_fall + season_winter + Rain + Snow + Fog + cluster_1 + cluster_2 + cluster_3 + cluster_4 + cluster_5 + cluster_6 + cluster_7'
# features = 'T0 + T1 + T2 + T3 + T4 + T5 + T6 + T7 + T8 + T9 + H0 + H1 + H2 + H3 + H4 + H5 + H6 + H7 + H8 + H9 + W0 + W1\
#             + season_spring + season_summer + season_fall + season_winter + Rain + Snow + Fog + cluster_1 + cluster_2 + cluster_3 + cluster_4 + cluster_5 + cluster_6 + cluster_7\
#             + month_1 + month_2 + month_3 + month_4 + month_5 + month_6 + month_7 + month_8 + month_9 + month_10 + month_11 + month_12'
#features = 'cluster_1 + cluster_2 + cluster_3 + cluster_4 + cluster_5 + cluster_6 + cluster_7 + season_spring + season_summer + season_fall + season_winter'
num = 216
lrdf

""


In [4]:
train_score_lst = []
test_score_lst = []
target_score_lst = []
name_lst = []
id_lst = []
dmTime = 0
elapsed = 0
probTime = 0

startTime = time()
X = dmatrix(features, all_info, return_type="dataframe")
tarX = dmatrix(features, target_info[num], return_type="dataframe")
endTime = time()
dmTime += endTime - startTime
for i in range(1, 629):
    target = 'Title.ID' + str(i)
    # y, X = dmatrices('Title_ID1 ~ meanT + avgH + avgW', all_info, return_type="dataframe")
    y = all_info[target]
    tarY = target_info[num][target]
    
    # flatten y into a 1-D array
    y = np.ravel(y)
#    print 'X.shape = %s  y.shape = %s' % (X.shape, y.shape)
    # model = LogisticRegression()
    # model = model.fit(X, y)
    # score_lst.append(model.score(X, y))
    # evaluate the model by splitting into train and test sets
    startTime = time()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    model2 = LogisticRegression()
    model2.fit(X_train, y_train)
    endTime = time()
    elapsed += endTime - startTime
    #train_score_lst.append(model2.predict_proba(X_train)[:,1])
    #test_score_lst.append(model2.predict_proba(X_test)[:,1])
    startTime = time()
    target_score_lst.append(model2.predict_proba(tarX)[:,1][0])
    endTime = time()
    probTime += endTime - startTime
    id_lst.append(i)
    

    # generate evaluation metrics
    title_name = freq_songs.loc[i-1].get_value('Title')
    name_lst.append(title_name)
    
print dmTime, elapsed, probTime

NameError: name 'features' is not defined

In [142]:
#lrdf['Score(train)'] = train_score_lst
#lrdf['Score(test)'] = test_score_lst
lrdf['Score(target)'] = target_score_lst
lrdf['Title'] = name_lst
lrdf['ID'] = id_lst

In [143]:
#train_rating = lrdf.sort_values('Score(train)', ascending=False)[:100]
#test_rating = lrdf.sort_values('Score(test)', ascending=False)[:100]
# 날짜별로 되어있는것을 하는것은 의미가 없음.
target_rating = lrdf.sort_values('Score(target)', ascending=False)[:100]
target_rating

,Score(target),Title,ID
61,0.326871,해변의 여인%by%쿨(COOL),62
268,0.287153,바다%by%유피(UP),269
239,0.276452,팥빙수%by%윤종신,240
2,0.248168,벌써 일년%by%브라운 아이즈(Brown Eyes),3
41,0.247497,벚꽃 엔딩%by%버스커 버스커(Busker Busker),42
187,0.241641,파도%by%유엔(UN),188
228,0.241622,여름아! 부탁해%by%인디고(Indigo),229
302,0.241460,여름안에서%by%서연,303
196,0.231274,Hot Summer%by%에프엑스(f(x)),197
13,0.223389,Never Ending Story%by%부활,14


In [144]:
target_chart[num]['name'] = target_chart[num].Title + '%by%' + target_chart[num].Artist
target_chart[num]

C:\Users\jinhyeok\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Rank,Title,Artist,Date,name
21585,1,UGLY,투애니원(2NE1),20120804,UGLY%by%투애니원(2NE1)
21586,2,달아요,박정현,20120804,달아요%by%박정현
21587,3,DOC와 춤을...,DJ DOC,20120804,DOC와 춤을...%by%DJ DOC
21588,4,아틀란티스 소녀,보아(BoA),20120804,아틀란티스 소녀%by%보아(BoA)
21589,5,Fiction,비스트(Beast),20120804,Fiction%by%비스트(Beast)
21590,6,Officially Missing You,긱스(Geeks),20120804,Officially Missing You%by%긱스(Geeks)
21591,7,아이스크림,MC 몽,20120804,아이스크림%by%MC 몽
21592,8,꿍따리 샤바라,클론(Clon),20120804,꿍따리 샤바라%by%클론(Clon)
21593,9,3!4!,룰라(Roo'Ra),20120804,3!4!%by%룰라(Roo'Ra)
21594,10,버스 안에서,자자(Zaza),20120804,버스 안에서%by%자자(Zaza)


In [145]:
t2 = target_chart[num]['name']
t1 = target_rating['Title']
t2.columns = ['Title']
#t3 = t1.append(t2)
#t3 = t1.intersection(t2)
t_intersect = pd.Series(list(set(t1) & set(t2)))
t_union = pd.Series(list(set(t1) | set(t2)))
JC_target = t_intersect.count() / float(t_union.count())
fm = freq_songs[:100]['Title']
fm_intersect = pd.Series(list(set(fm) & set(t2)))
fm_union = pd.Series(list(set(fm) | set(t2)))
JC_fm = fm_intersect.count() / float(fm_union.count())
print JC_fm, JC_target

0.0526315789474 0.111111111111


In [28]:
t_union.count()

186

In [25]:
all_info

,Unnamed: 0,Date,meanT,maxT,minT,dewPoint,avgH,maxH,minH,pressure,...,month_7,month_8,month_9,month_10,month_11,month_12,season_fall,season_spring,season_summer,season_winter
0,0,2012-01-01,-4,0,-8,-9,64,80,37,1024.69,...,0,0,0,0,0,0,0,0,0,1
1,1,2012-01-02,-6,-1,-11,-12,63,79,47,1026.96,...,0,0,0,0,0,0,0,0,0,1
2,2,2012-01-03,-4,1,-9,-11,68,80,35,1022.16,...,0,0,0,0,0,0,0,0,0,1
3,3,2012-01-04,-6,-3,-10,-16,52,68,31,1025.75,...,0,0,0,0,0,0,0,0,0,1
4,4,2012-01-05,-6,-1,-12,-15,55,79,25,1029.50,...,0,0,0,0,0,0,0,0,0,1
5,5,2012-01-06,-2,1,-8,-14,45,64,20,1031.00,...,0,0,0,0,0,0,0,0,0,1
6,6,2012-01-07,-5,1,-11,-14,54,73,28,1035.21,...,0,0,0,0,0,0,0,0,0,1
7,7,2012-01-08,-4,1,-9,-11,62,80,38,1032.29,...,0,0,0,0,0,0,0,0,0,1
8,8,2012-01-09,-1,2,-4,-7,66,86,48,1030.12,...,0,0,0,0,0,0,0,0,0,1
9,9,2012-01-10,-4,2,-9,-11,56,80,35,1028.29,...,0,0,0,0,0,0,0,0,0,1


# Jacard File Save

In [ ]:
all_info = pd.read_csv('160205_cluster/integrated_0or1_larger_sect_with_cluster.csv', encoding='utf-8-sig')
freq_songs = pd.read_csv('160204_advanced/freqSongs.csv', encoding='utf-8-sig')

normal_info = DataFrame({})

df = all_info.copy()
df = df.select_dtypes(exclude=['object'])
#df.events
normal_info = ( df - all_info.mean() ) / all_info.std()
normal_info = normal_info[['meanT', 'maxT', 'minT', 'dewPoint',
                           'avgH', 'maxH', 'minH', 'avgW',
                           'maxW', 'visibleD', 'pressure', 'wind_chill',
                           'heat_index', 'discomfort_index']] # 14 column

all_info.drop(all_info.columns[[1,2,3,4,5,6,7,8,9,10,11,14,15,16]], axis=1, inplace=True)

# Get Logistic data & Join column
month_logit = pd.get_dummies(all_info['Month'], prefix='month') # 12 column
season_logit = pd.get_dummies(all_info['Season'], prefix='season') # 4 column
cluster_logit = pd.get_dummies(all_info['cluster'], prefix='cluster') # 8 column

print all_info.shape
all_info = all_info.join(month_logit)
all_info = all_info.join(season_logit)
all_info = all_info.join(normal_info)
all_info = all_info.join(cluster_logit)
all_info['cnt'] = 1
print all_info.shape
all_info

In [ ]:
target_info = []
target_chart = []
target_date = date(2012, 1, 1)
for i in range(1461):
    target_date_str = target_date.strftime('%Y-%m-%d')
    target_date_int = target_date.year * 10000 + target_date.month * 100 + target_date.day
    
    target_info.append(all_info[all_info.Date == target_date_str])
    target_chart.append(mania_chart[mania_chart.Date == target_date_int])
    target_date = target_date + td
    

In [11]:
target_info[1460]

,Date,events,highlight,Title.ID1,Title.ID2,Title.ID3,Title.ID4,Title.ID5,Title.ID6,Title.ID7,...,discomfort_index,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cnt
1457,2015-12-31,None,"Waning Gibbous, 69",0,0,0,0,0,1,1,...,-1.320955,1,0,0,0,0,0,0,0,1


In [ ]:
#features = 'meanT'
#features = 'meanT + avgH + avgW'
#features = 'meanT + avgH + avgW + Season + Rain'
#features = 'meanT + avgH + avgW + Rain + Month'
#features = 'T0 + T1 + T2 + T3 + T4 + T5 + T6 + T7 + T8 + T9 + H0 + H1 + H2 + H3 + H4 + H5 + H6 + H7 + H8 + H9 + W0 + W1 + Rain + Snow + Month + heat_index + discomfort_index'
#features = 'T0 + T1 + T2 + T3 + T4 + T5 + T6 + T7 + T8 + T9 + H0 + H1 + H2 + H3 + H4 + H5 + H6 + H7 + H8 + H9 + W0 + W1 + season_spring + season_summer + season_fall + season_winter'
features = 'T0 + T1 + T2 + T3 + T4 + T5 + T6 + T7 + T8 + T9 + H0 + H1 + H2 + H3 + H4 + H5 + H6 + H7 + H8 + H9 + W0 + W1\
            + season_spring + season_summer + season_fall + season_winter + Rain + Snow + Fog\
            + cluster_1 + cluster_2 + cluster_3 + cluster_4 + cluster_5 + cluster_6 + cluster_7 + cluster_8'

jc = {}
JacardDF = DataFrame(jc)
JC_fm_lst = []
JC_target_lst = []
intersect_lst = []
cur_date_lst = []

cur_date = date(2012, 1, 1)
fm = freq_songs[:100]['Title']

X = dmatrix(features, all_info, return_type="dataframe")

for j in range(1461):
    try:
        target_score_lst = []
        name_lst = []
        id_lst = []
        temp = {}
        lrdf = DataFrame(temp)
        
        tarX = dmatrix(features, target_info[j], return_type="dataframe")
        
        for i in range(1, 629):
            target = 'Title.ID' + str(i)
            y = all_info[target]
            tarY = target_info[j][target]
            
            # flatten y into a 1-D array
            y = np.ravel(y)
        #    print 'X.shape = %s  y.shape = %s' % (X.shape, y.shape)
            # model = LogisticRegression()
            # model = model.fit(X, y)
            # score_lst.append(model.score(X, y))
            # evaluate the model by splitting into train and test sets
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
            model2 = LogisticRegression()
            model2.fit(X_train, y_train)
            #train_score_lst.append(model2.predict_proba(X_train)[:,1])
            #test_score_lst.append(model2.predict_proba(X_test)[:,1])
            target_score_lst.append(model2.predict_proba(tarX)[:,1][0])
            id_lst.append(i)

            # generate evaluation metrics
            title_name = freq_songs.loc[i-1].get_value('Title')
            name_lst.append(title_name)

        lrdf['Score(target)'] = target_score_lst
        lrdf['Title'] = name_lst
        lrdf['ID'] = id_lst

        target_rating = lrdf.sort_values('Score(target)', ascending=False)[:100]

        target_chart[j]['name'] = target_chart[j].Title + '%by%' + target_chart[j].Artist

        t2 = target_chart[j]['name']
        t1 = target_rating['Title']
        t2.columns = ['Title']
        #t3 = t1.append(t2)
        #t3 = t1.intersection(t2)
        t_intersect = pd.Series(list(set(t1) & set(t2)))
        t_union = pd.Series(list(set(t1) | set(t2)))
        JC_target = t_intersect.count() / float(t_union.count())

        fm_intersect = pd.Series(list(set(fm) & set(t2)))
        fm_union = pd.Series(list(set(fm) | set(t2)))
        JC_fm = fm_intersect.count() / float(fm_union.count())
        
        last_intersect = pd.Series(list(set(t_intersect) & set(fm_intersect)))
        last_union = pd.Series(list(set(fm_intersect) | set(t_intersect)))
        JC_last = last_intersect.count() / float(last_union.count())
        print str(cur_date) + '==============='
        print JC_fm, JC_target
        #print 'Original Rank ============'
        #print fm_intersect
        #print 'Regression Rank ============'
        #print t_intersect
        print 'cross ========'
        print JC_last
        print last_intersect
        JC_fm_lst.append(JC_fm)
        JC_target_lst.append(JC_target)
        intersect_lst.append(last_intersect.count())
        origin_correct_lst.append(fm_intersect.count())
        correct_lst.append(t_intersect.count())
        cur_date_lst.append(str(cur_date))

        cur_date = cur_date + td
    except:
        print "There is not " + str(cur_date)
        cur_date = cur_date + td
        continue

JacardDF['Original'] = JC_fm_lst
JacardDF['Logit'] = JC_target_lst
JacardDF['O_Correct'] = origin_correct_lst
JacardDF['L_Correct'] = correct_lst
JacardDF['noIntersect'] = intersect_lst
JacardDF['Date'] = cur_date_lst

2012-01-01===============
0.0 0.0
cross ========
nan
Series([], dtype: float64)
2012-01-02===============
0.0 0.0
cross ========
nan
Series([], dtype: float64)
2012-01-03===============
0.0 0.0
cross ========
nan
Series([], dtype: float64)
2012-01-04===============
0.0 0.0
cross ========
nan
Series([], dtype: float64)
There is not 2012-01-05
2012-01-06===============
0.0 0.0
cross ========
nan
Series([], dtype: float64)
2012-01-07===============

In [160]:
JacardDF.describe()

,Original,Logit,noIntersect
count,1458.000000,1458.000000,1458.000000
mean,0.072125,0.113248,9.046639
std,0.025587,0.048087,3.704495
min,0.010101,0.015228,1.000000
25%,0.052910,0.081081,6.000000
50%,0.069519,0.104972,9.000000
75%,0.086957,0.142857,11.000000
max,0.156069,0.283871,22.000000


In [161]:
JacardDF.to_csv('Jacard_ALLINONE_4.csv', encoding='utf-8-sig')

# Jaccard Test set 

In [100]:
features = 'T0 + T1 + T2 + T3 + T4 + T5 + T6 + T7 + T8 + T9 + H0 + H1 + H2 + H3 + H4 + H5 + H6 + H7 + H8 + H9 + W0 + W1\
            + season_spring + season_summer + season_fall + season_winter + heat_index + wind_chill\
            + cluster_1 + cluster_2 + cluster_3 + cluster_4 + cluster_5 + cluster_6 + cluster_7 + cluster_8'
# features = 'cluster_1 + cluster_2 + cluster_3 + cluster_4 + cluster_5 + cluster_6 + cluster_7 + cluster_8'
# features = 'cluster_1 + cluster_2 + cluster_3 + cluster_4 + cluster_5 + cluster_6 + cluster_7 + cluster_8\
#             + heat_index + wind_chill'


In [101]:
y = all_info['Title.ID1']
ainfo_train, ainfo_test, y_train, y_test = train_test_split(all_info, y, test_size=0.3, random_state=0)
ainfo_test.describe()
X_train = dmatrix(features, ainfo_train, return_type="dataframe")
X_test = dmatrix(features, ainfo_test, return_type="dataframe")
y_train = ainfo_train[ainfo_train.columns[[range(3,631)]]]
y_test = ainfo_test[ainfo_test.columns[[range(3,631)]]]

y_train.describe()

,Title.ID1,Title.ID2,Title.ID3,Title.ID4,Title.ID5,Title.ID6,Title.ID7,Title.ID8,Title.ID9,Title.ID10,...,Title.ID619,Title.ID620,Title.ID621,Title.ID622,Title.ID623,Title.ID624,Title.ID625,Title.ID626,Title.ID627,Title.ID628
count,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,...,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000
mean,0.213725,0.210784,0.219608,0.199020,0.188235,0.185294,0.190196,0.178431,0.184314,0.158824,...,0.033333,0.036275,0.026471,0.033333,0.030392,0.041176,0.033333,0.032353,0.039216,0.030392
std,0.410136,0.408066,0.414184,0.399459,0.391092,0.388727,0.392648,0.383063,0.387930,0.365691,...,0.179594,0.187064,0.160609,0.179594,0.171748,0.198796,0.179594,0.177022,0.194203,0.171748
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [102]:
jc = {}
JaccardDF = DataFrame(jc)
JC_fm_lst = []
JC_target_lst = []
intersect_lst = []
cur_date_lst = []
origin_correct_lst = []
correct_lst = []
JC_OnL_lst = []

cur_date = date(2012, 1, 1)
fm = freq_songs[:100]['Title']

model_lst = []

for i in range(628):
    model = LogisticRegression() 
    model.fit(X_train, y_train[y_train.columns[i]])
#     test_score_lst.append(model_lst[i-1].predict_proba(X_test)[:,1])
    model_lst.append(model)

for j in range(1461):
    try:
        target_data = ainfo_test[ainfo_test['Date'] == target_info[j]['Date'].get_values()[0]]
        if not target_data.shape[0]:
            cur_date = cur_date + td
            continue

        target_score_lst = []
        name_lst = []
        id_lst = []
        temp = {}
        lrdf = DataFrame(temp)

        tarX = dmatrix(features, target_info[j], return_type="dataframe")

        for i in range(628):
    #             train_score_lst.append(model2.predict_proba(X_train)[:,1])
    #             test_score_lst.append(model_lst[i-1].predict_proba(X_test)[:,1])
            target_score_lst.append(model_lst[i].predict_proba(tarX)[:,1][0])
            id_lst.append(i)

            # generate evaluation metrics
            title_name = freq_songs.loc[i].get_value('Title')
            name_lst.append(title_name)

        lrdf['Score(target)'] = target_score_lst
        lrdf['Title'] = name_lst
        lrdf['ID'] = id_lst

        target_rating = lrdf.sort_values('Score(target)', ascending=False)[:100]

        target_chart[j]['name'] = target_chart[j].Title + '%by%' + target_chart[j].Artist

        t2 = target_chart[j]['name']
        t1 = target_rating['Title']
        t2.columns = ['Title']
        #t3 = t1.append(t2)
        #t3 = t1.intersection(t2)
        t_intersect = pd.Series(list(set(t1) & set(t2)))
        t_union = pd.Series(list(set(t1) | set(t2)))
        JC_target = t_intersect.count() / float(t_union.count())

        fm_intersect = pd.Series(list(set(fm) & set(t2)))
        fm_union = pd.Series(list(set(fm) | set(t2)))
        JC_fm = fm_intersect.count() / float(fm_union.count())

        last_intersect = pd.Series(list(set(t_intersect) & set(fm_intersect)))
        last_union = pd.Series(list(set(fm_intersect) | set(t_intersect)))
        JC_last = last_intersect.count() / float(last_union.count())
        print str(cur_date) + '==============='
        print 'Recomm Correct: ', t_intersect.count()
        print JC_fm, JC_target
        #print 'Original Rank ============'
        #print fm_intersect
        #print 'Regression Rank ============'
        #print t_intersect
        print 'cross ========'
        print JC_last
    #     print last_intersect
        JC_fm_lst.append(JC_fm)
        JC_target_lst.append(JC_target)
        intersect_lst.append(last_intersect.count())
        origin_correct_lst.append(fm_intersect.count())
        JC_OnL_lst.append(JC_last)
        correct_lst.append(t_intersect.count())
        cur_date_lst.append(str(cur_date))

        cur_date = cur_date + td
    except IndexError as e:
        print "There is not " + str(cur_date)
        cur_date = cur_date + td
        continue

JaccardDF['JC_Original'] = JC_fm_lst
JaccardDF['JC_Logit'] = JC_target_lst
JaccardDF['O_Correct'] = origin_correct_lst
JaccardDF['L_Correct'] = correct_lst
JaccardDF['JC_OnL'] = JC_OnL_lst
JaccardDF['noIntersect'] = intersect_lst
JaccardDF['Date'] = cur_date_lst

2012-01-02===============
Recomm Correct:  8
0.0810810810811 0.0416666666667
cross ========
0.352941176471
2012-01-03===============
Recomm Correct:  9
0.0752688172043 0.0471204188482
cross ========
0.277777777778
2012-01-05===============
Recomm Correct:  15
0.0695187165775 0.0810810810811
cross ========
0.272727272727
2012-01-06===============
Recomm Correct:  6
0.0471204188482 0.0309278350515
cross ========
0.25
2012-01-09===============
Recomm Correct:  8
0.0416666666667 0.0416666666667
cross ========
0.333333333333
2012-01-10===============
Recomm Correct:  10
0.0362694300518 0.0526315789474
cross ========
0.214285714286
2012-01-11===============
Recomm Correct:  9
0.0416666666667 0.0471204188482
cross ========
0.307692307692
2012-01-15===============
Recomm Correct:  15
0.0309278350515 0.0810810810811
cross ========
0.166666666667
2012-01-19===============
Recomm Correct:  5
0.025641025641 0.025641025641
cross ========
0.25
2012-01-20===============
Recomm Correct:  3
0.020408163

C:\Users\jinhyeok\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [104]:
JaccardDF.describe()

,JC_Original,JC_Logit,O_Correct,L_Correct,JC_OnL,noIntersect
count,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000
mean,0.071809,0.087051,13.280822,15.767123,0.370444,7.828767
std,0.025398,0.038947,4.384928,6.445733,0.120621,3.355435
min,0.015228,0.015228,3.000000,3.000000,0.045455,1.000000
25%,0.053191,0.058201,10.000000,11.000000,0.277778,5.000000
50%,0.069519,0.081522,13.000000,15.000000,0.361818,8.000000
75%,0.086957,0.111111,16.000000,20.000000,0.450000,10.000000
max,0.150289,0.234568,26.000000,38.000000,0.714286,20.000000


In [16]:
JaccardDF.to_csv('Jaccard_CSV/Jaccard_CLUSTER_test03.csv', encoding='utf-8-sig')

In [95]:
discrete = JaccardDF[['O_Correct', 'L_Correct', 'Date']]
discrete.columns = ['NullModel', 'LogisticRegression', 'Date']
discrete['Date'] = pd.to_datetime(discrete['Date'], format='%Y-%m-%d')
dfd = discrete.groupby('Date').mean()
dg = discrete.groupby(dfd.index.month).mean()
ax = dg.plot(kind='bar', alpha=0.7, fontsize=12)
ax.set_title('The Average of Number of the Monthly Correct Answers', fontsize=18)
ax.set_xlabel('Month', fontsize=15)
ax.set_ylabel('The Average of Number of Correct Answers', fontsize=15)
ax.title.set_position([.5, 1.05])

C:\Users\jinhyeok\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [86]:
fig.savefig('asa.jpg')


# Collaborative Filtering - Alternative Least Square

In [6]:
mania_chart = pd.read_csv('160130/mania_data_revised.csv', encoding='utf-8-sig')
freq_songs = pd.read_csv('160204_advanced/freqSongs.csv', encoding='utf-8-sig')
weathers = pd.read_csv('160205_cluster/integrated_0or1_larger_sect_with_cluster.csv')

d = weathers.Date.astype(str)
for i in range(len(d)):
    dstr = d[i].split('-')
    dstr = map(int, dstr)
    d[i] = dstr[0] * 10000 + dstr[1] * 100 + dstr[2]
weathers.Date = d
mania_chart['cnt'] = 1
mania_chart['Title'] = mania_chart['Title'] + '%by%' + mania_chart['Artist']

In [7]:
LIMIT_FREQ = 30

In [5]:
main_songs = freq_songs[freq_songs['freq'] >= LIMIT_FREQ]
main_songs.freq = ( main_songs.freq - main_songs.freq.mean() ) / main_songs.freq.std()
mf = main_songs.pivot_table(values='freq', columns=['Title'], aggfunc=np.sum)
mf

C:\Users\jinhyeok\AppData\Local\Enthought\Canopy\User\lib\site-packages\pandas\core\generic.py:2387: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


Title
('Cause) I'm Your Girl%by%S.E.S.                         -0.537713
...사랑했잖아...%by%린(LYn)                                     0.734948
...안 되나요... (부제: 화양연화)%by%휘성                              0.359409
1,2,3,4 (원,투,쓰리,포)%by%이하이                                -0.099584
10월(月)에 눈이 내리면%by%성시경                                    -0.829799
12월 32일%by%별                                              0.359409
12월%by%루싸이트 토끼                                           -0.829799
12월의 드라마 (The Drama Of December)%by%디바(Diva)             -0.829799
17171771%by%자우림(Jaurim)                                  -0.537713
1994년 어느 늦은 밤%by%장혜진                                      0.046460
1999 사랑 (부제: 흑야애(黑夜愛))%by%제이벨(J BEL)                     -0.642029
1분 1초 (feat. 타루)%by%에픽 하이(Epik High)                     -0.537713
1월부터 6월까지 (feat. 윤종신)%by%공일오비(015B)                       0.234229
200%%by%악동뮤지션(AKMU)                                      -0.767209
2HOT%by%지나(G.NA)                                        

In [6]:
cnt_for_title = mania_chart.groupby(['Title']).sum().sort_values('cnt', ascending=False)
main_songs = cnt_for_title[cnt_for_title.cnt >= LIMIT_FREQ]
del main_songs['Date']
del main_songs['Rank']
#df = pd.merge(mania_chart, main_songs)
#mania_chart[mania_chart.inmain_songs.index]
main_songs_title_lst = list(main_songs.index.get_level_values(0))
# Main Song 에 대한 차트만 가져오기
main_chart = mania_chart[mania_chart.isin({'Title':main_songs_title_lst}).Title]

In [7]:
for i in range(10):
    T_col_name = 'T' + str(i)
    H_col_name = 'H' + str(i)
    weathers.ix[weathers[T_col_name] == 1, 'Tsection'] = i//2
    weathers.ix[weathers[H_col_name] == 1, 'Hsection'] = i//2
for i in range(2):
    W_col_name = 'W' + str(i)
    weathers.ix[weathers[W_col_name] == 1, 'Wsection'] = i+1
weathers

,Date,meanT,maxT,minT,dewPoint,avgH,maxH,minH,pressure,avgW,...,H5,H6,H7,H8,H9,W0,W1,Tsection,Hsection,Wsection
0,20120101,-4,0,-8,-9,64,80,37,1024.69,9,...,1,0,0,0,0,0,1,0,2,2
1,20120102,-6,-1,-11,-12,63,79,47,1026.96,5,...,0,0,0,0,0,1,0,0,2,1
2,20120103,-4,1,-9,-11,68,80,35,1022.16,6,...,0,1,0,0,0,1,0,0,3,1
3,20120104,-6,-3,-10,-16,52,68,31,1025.75,11,...,0,0,0,0,0,0,1,0,1,2
4,20120105,-6,-1,-12,-15,55,79,25,1029.50,4,...,0,0,0,0,0,1,0,0,1,1
5,20120106,-2,1,-8,-14,45,64,20,1031.00,7,...,0,0,0,0,0,1,0,0,0,1
6,20120107,-5,1,-11,-14,54,73,28,1035.21,4,...,0,0,0,0,0,1,0,0,1,1
7,20120108,-4,1,-9,-11,62,80,38,1032.29,4,...,0,0,0,0,0,1,0,0,2,1
8,20120109,-1,2,-4,-7,66,86,48,1030.12,3,...,1,0,0,0,0,1,0,0,2,1
9,20120110,-4,2,-9,-11,56,80,35,1028.29,6,...,0,0,0,0,0,1,0,0,1,1


In [8]:
weathers['wcnt'] = 1
weathers_train, weathers_test, y_train, y_test = train_test_split(weathers, weathers.wcnt, test_size=0.2, random_state=0) # y 무효
weathers.groupby(['cluster', 'Season']).sum() # wcnt 파악

meanT  maxT  minT  dewPoint   avgH   maxH  minH   pressure  \
cluster Season                                                               
1       fall      102   250   -51      -188   1687   2377   979   31729.53   
        spring    186   544  -168      -593   3045   4531  1561   67466.40   
        winter   -643   486 -1790     -2849  12341  16915  7276  240101.06   
2       fall      603   689   512       456   2250   2656  1748   30382.41   
        spring    335   395   277       229   1374   1666  1006   19134.66   
        summer   2508  2801  2211      1978   7608   8984  5971  101538.47   
3       fall      230   282   172       147   1174   1422   849   16212.18   
        spring    146   191   101        73    564    751   372    9080.24   
        summer   1146  1281  1005       940   3676   4159  2897   46280.81   
4       fall     1654  2377   936       599   6728   9651  3500  117316.57   
        spring   2025  3013  1042       308   7352  11384  3501  148233.82   
        summer    194   253   135        72    412    673   190    9096.65   
        winter     15    37    -5       -10    172    254    89    3058.35   
5       fall      499   709   294       301   2700   3381  1527   36699.77   
        spring    366   551   178       144   1967   2631   987   29408.74   
        summer    201   241   162       154    599    758   387    8053.03   
        winter     55   188   -91       -91   2255   2704  1432   30623.36   
6       fall      462   638   277       192   3434   4328  2352   49890.57   
        spring    583   827   322       105   3823   5095  2391   61740.78   
        winter     60   135   -25       -80   1435   1845   905   22430.51   
7       fall       21    55   -15       -49    625    799   432   10176.80   
        spring     13    37    -9       -41    343    509   174    6093.07   
        winter   -223    91  -538      -682   4528   5899  2934   73615.67   
8       fall     1603  1978  1225      1053   5112   6697  3173   77077.21   
        spring    606   765   447       322   1857   2469  1174   31292.47   
        summer   5169  6069  4268      3679  13569  17477  9345  204534.98   

                avgW  maxW  ...   H6  H7  H8  H9   W0  W1  Tsection  Hsection  \
cluster Season              ...                                                 
1       fall     209   513  ...    0   1   0   0   21  10        24        29   
        spring   674  1500  ...    1   0   0   0   20  46        51        31   
        winter  1790  4062  ...   16   6   1   0  147  87        42       237   
2       fall     237   571  ...    3   4   9   9   14  16        83       101   
        spring   170   419  ...    4   1   3   5    8  11        45        58   
        summer   976  2116  ...   10  11  27  36   47  54       357       341   
3       fall     162   417  ...    0   2   1   7    4  12        32        48   
        spring   116   281  ...    2   0   1   0    1   8        20        19   
        summer   463  1173  ...    3   2  14  25   14  32       164       175   
4       fall     799  1880  ...   12   6   4   0   75  40       239       171   
        spring  1128  2957  ...    5   2   0   0   78  68       292        99   
        summer    69   186  ...    0   0   0   0    6   3        27         2   
        winter    23    58  ...    0   0   0   0    2   1         3         4   
5       fall     188   525  ...    4   9  11   9   33   3        69       125   
        spring   217   586  ...    4   5   6   2   18  11        55        78   
        summer    47   136  ...    0   2   5   0    8   0        29        28   
        winter   175   472  ...    5   6   4  11   25   5        19       100   
6       fall     489  1097  ...   12   2   9  10   17  32        75       141   
        spring   680  1498  ...    7   7   8   3   13  48        91       122   
        winter   205   461  ...    2   1   2   4    8  14        16        49   
7       fall     111   246  ...    0   1   1   0 

In [9]:
df = pd.merge(main_chart, weathers_train)
# df = pd.merge(main_chart, weathers)
#one = df.groupby(['Title', 'Artist'], as_index=False).mean()
#one = df.groupby(['cluster'], as_index=False).mean()
pt = df.pivot_table(values='cnt', index=['Title'], columns=['cluster', 'Season'], aggfunc=np.sum)
pt = pt.fillna(0)
rp = pt.T
rp
crp = rp.copy()
crp = crp.T
ncrp = (crp - crp.mean()) / crp.std()
rp = ncrp.T
rp

Title           ('Cause) I'm Your Girl%by%S.E.S.  ...사랑했잖아...%by%린(LYn)  \
cluster Season                                                            
1       fall                           -0.581103               2.032336   
        spring                          1.254258               0.758590   
        winter                          0.050342              -0.072092   
2       fall                           -0.676219               4.342404   
        spring                          0.179886               1.898475   
        summer                         -0.661886              -0.201560   
3       fall                           -0.633870               1.456942   
        spring                         -0.523080               1.010349   
        summer                         -0.765629              -0.765629   
4       fall                           -0.833592              -0.642961   
        spring                          0.398701               0.221254   
        summer                         -0.344749               1.681498   
        winter                         -0.307018              -0.307018   
5       fall                           -0.725995               0.483828   
        spring                         -0.183006               0.968825   
        summer                         -0.512282               1.208238   
        winter                         -0.755471              -0.755471   
6       fall                           -0.697151               0.134161   
        spring                         -0.474318              -0.120644   
        winter                          0.938813               0.938813   
7       fall                           -0.427658               0.904192   
        spring                          1.437889               1.437889   
        winter                         -0.139260              -0.507853   
8       fall                           -0.527810               2.636413   
        spring                         -0.606598              -0.111492   
        summer                         -0.935879              -0.236304   

Title           ...안 되나요... (부제: 화양연화)%by%휘성  1,2,3,4 (원,투,쓰리,포)%by%이하이  \
cluster Season                                                            
1       fall                        0.986960                   0.464272   
        spring                     -0.728412                   0.015089   
        winter                      0.662509                   0.050342   
2       fall                       -0.118594                  -0.676219   
        spring                      2.757769                  -0.679409   
        summer                     -0.431723                   0.028602   
3       fall                        0.411536                   0.411536   
        spring                     -0.523080                  -0.523080   
        summer                     -0.765629                   0.231643   
4       fall                        0.691454                  -0.642961   
        spring                      0.576148                   0.931042   
        summer                     -0.344749                  -0.344749   
        winter                      2.710314                  -0.307018   
5       fall                        0.483828                   2.298562   
        spring                     -0.183006                  -0.183006   
        summer                     -0.512282                  -0.512282   
        winter                      0.411058                  -0.755471   
6       fall                       -0.281495                   0.965472   
        spring                     -0.827992                  -0.827992   
        winter                      1.745722                  -0.675003   
7       fall                        0.904192                   2.236043   
        spring                     -0.521235                  -0.521235   
        winter                      0.229333                  -0.139260   
8       fall            

In [10]:
Q = rp.values
Q

array([[-0.58110335,  2.03233595,  0.98696023, ..., -0.58110335,
         0.98696023,  2.5550238 ],
       [ 1.25425768,  0.75859026, -0.72841201, ..., -0.72841201,
        -0.72841201, -0.72841201],
       [ 0.05034167, -0.07209172,  0.66250863, ..., -0.56182529,
        -0.07209172,  0.66250863],
       ..., 
       [-0.52781031,  2.63641251,  0.79061586, ..., -0.26412508,
        -0.79149555, -0.79149555],
       [-0.60659803, -0.11149181,  2.85914551, ..., -0.60659803,
         1.86893307, -0.60659803],
       [-0.9358793 , -0.23630369, -0.5860915 , ..., -0.23630369,
        -0.23630369, -0.9358793 ]])

In [11]:
ran_music_lst = np.random.choice(628, 100, replace=False)
ran_cluster_lst = np.random.choice(8, 100)
ran_total_lst = zip(ran_cluster_lst, ran_music_lst)
ran_total_lst

[(2, 353),
 (7, 392),
 (0, 195),
 (2, 465),
 (5, 277),
 (2, 149),
 (5, 348),
 (0, 153),
 (0, 600),
 (2, 519),
 (0, 189),
 (4, 337),
 (7, 279),
 (5, 81),
 (6, 377),
 (5, 336),
 (4, 468),
 (0, 152),
 (1, 598),
 (5, 462),
 (3, 419),
 (6, 523),
 (0, 169),
 (6, 249),
 (4, 323),
 (0, 159),
 (1, 39),
 (4, 476),
 (6, 389),
 (0, 217),
 (5, 114),
 (1, 304),
 (2, 40),
 (1, 9),
 (0, 78),
 (5, 442),
 (6, 58),
 (0, 379),
 (1, 627),
 (1, 335),
 (7, 209),
 (4, 493),
 (0, 137),
 (5, 524),
 (0, 188),
 (1, 457),
 (4, 405),
 (0, 76),
 (7, 560),
 (4, 254),
 (3, 117),
 (4, 18),
 (3, 226),
 (4, 580),
 (4, 542),
 (6, 285),
 (1, 517),
 (1, 536),
 (4, 264),
 (2, 479),
 (6, 548),
 (1, 494),
 (7, 192),
 (1, 541),
 (6, 472),
 (7, 201),
 (2, 431),
 (3, 287),
 (4, 176),
 (5, 100),
 (4, 501),
 (5, 52),
 (7, 238),
 (6, 87),
 (4, 53),
 (3, 133),
 (3, 426),
 (5, 619),
 (1, 24),
 (5, 44),
 (2, 215),
 (0, 485),
 (7, 361),
 (7, 421),
 (6, 84),
 (2, 372),
 (1, 594),
 (5, 29),
 (7, 36),
 (5, 599),
 (0, 496),
 (6, 562),
 (7, 

In [12]:
W = Q > -sys.maxint
W[W == True] = 1
# for ran_cluster, ran_music in ran_total_lst:
#     W[ran_cluster][ran_music] = 0
# W = Q>0.5
# W[W == True] = 1
# W[W == False] = 0
W = W.astype(np.float64, copy=False)
W

array([[ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       ..., 
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.]])

In [13]:
print Q.shape
lambda_ = 0.1
n_factors = 100
m, n = Q.shape # (8, 628)
n_iterations = 20

(26L, 1199L)


In [14]:
X = 0.1 * np.random.rand(m, n_factors) # (8, 100)
Y = 0.1 * np.random.rand(n_factors, n) # (100, 628)

In [15]:
def get_error(Q, X, Y, W):
    return np.sum((W *(Q - np.dot(X, Y))) ** 2)

In [16]:
errors = []
for ii in range(n_iterations):
    X = np.linalg.solve(np.dot(Y, Y.T) + lambda_ * np.eye(n_factors),
                       np.dot(Y, Q.T)).T
    Y = np.linalg.solve(np.dot(X.T, X) + lambda_ * np.eye(n_factors),
                       np.dot(X.T, Q))
    print('{}th iteration is completed'.format(ii))
    errors.append(get_error(Q, X, Y, W))
Q_hat = np.dot(X, Y)
print('Error of evented musics: {}'.format(get_error(Q, X, Y, W)))

0th iteration is completed
1th iteration is completed
2th iteration is completed
3th iteration is completed
4th iteration is completed
5th iteration is completed
6th iteration is completed
7th iteration is completed
8th iteration is completed
9th iteration is completed
10th iteration is completed
11th iteration is completed
12th iteration is completed
13th iteration is completed
14th iteration is completed
15th iteration is completed
16th iteration is completed
17th iteration is completed
18th iteration is completed
19th iteration is completed
Error of evented musics: 0.178260415715


In [17]:
plt.plot(errors)
plt.ylim([0, 10])
plt.xlabel('n iteration')
plt.ylabel('error')
print errors

[0.20186358152683487, 0.19964691131374318, 0.19757883859109587, 0.19564999776598374, 0.19385174810292646, 0.19217610512762362, 0.19061567964757611, 0.18916362342635845, 0.18781358068460721, 0.1865596447156696, 0.185396319000876, 0.18431848229231454, 0.18332135720127551, 0.18240048189063418, 0.18155168452079712, 0.18077106014349176, 0.1800549497750738, 0.17939992141448763, 0.17880275279891478, 0.17826041571512952]


In [18]:
weighted_errors = []
for ii in range(n_iterations):
    for u, Wu in enumerate(W):
        X[u] = np.linalg.solve(np.dot(Y, np.dot(np.diag(Wu), Y.T)) + lambda_ * np.eye(n_factors),
                               np.dot(Y, np.dot(np.diag(Wu), Q[u].T))).T
    for i, Wi in enumerate(W.T):
        Y[:,i] = np.linalg.solve(np.dot(X.T, np.dot(np.diag(Wi), X)) + lambda_ * np.eye(n_factors),
                                 np.dot(X.T, np.dot(np.diag(Wi), Q[:, i])))
    weighted_errors.append(get_error(Q, X, Y, W))
    print('{}th iteration is completed'.format(ii))
weighted_Q_hat = np.dot(X,Y)
#print('Error of rated movies: {}'.format(get_error(Q, X, Y, W)))

0th iteration is completed
1th iteration is completed
2th iteration is completed
3th iteration is completed
4th iteration is completed
5th iteration is completed
6th iteration is completed
7th iteration is completed
8th iteration is completed
9th iteration is completed
10th iteration is completed
11th iteration is completed
12th iteration is completed
13th iteration is completed
14th iteration is completed
15th iteration is completed
16th iteration is completed
17th iteration is completed
18th iteration is completed
19th iteration is completed


In [19]:
plt.plot(weighted_errors);
plt.xlabel('Iteration Number');
plt.ylabel('Mean Squared Error');
print weighted_errors

[0.17777006170570339, 0.17732900902786181, 0.17693473073912436, 0.17658484379793479, 0.17627709908001082, 0.17600937222182556, 0.17577965521250616, 0.17558604866367938, 0.17542675469418575, 0.17530007037332621, 0.17520438167204111, 0.17513815787639939, 0.17509994642271171, 0.17508836811727779, 0.17510211270764092, 0.1751399347752986, 0.17520064992288575, 0.17528313123107078, 0.1753863059631996, 0.17550915249732252]


In [20]:
print Q
print Q_hat
# print weighted_Q_hat

[[-0.58110335  2.03233595  0.98696023 ..., -0.58110335  0.98696023
   2.5550238 ]
 [ 1.25425768  0.75859026 -0.72841201 ..., -0.72841201 -0.72841201
  -0.72841201]
 [ 0.05034167 -0.07209172  0.66250863 ..., -0.56182529 -0.07209172
   0.66250863]
 ..., 
 [-0.52781031  2.63641251  0.79061586 ..., -0.26412508 -0.79149555
  -0.79149555]
 [-0.60659803 -0.11149181  2.85914551 ..., -0.60659803  1.86893307
  -0.60659803]
 [-0.9358793  -0.23630369 -0.5860915  ..., -0.23630369 -0.23630369
  -0.9358793 ]]
[[-0.58178966  2.02420326  0.98074622 ..., -0.58131647  0.98245079
   2.54960238]
 [ 1.25219541  0.75740472 -0.72589166 ..., -0.72776631 -0.72775525
  -0.72577401]
 [ 0.04823896 -0.07060452  0.66328929 ..., -0.5602647  -0.07097387
   0.66470604]
 ..., 
 [-0.52813302  2.63610678  0.78713274 ..., -0.26508237 -0.79195655
  -0.78967564]
 [-0.60478436 -0.10768279  2.85490105 ..., -0.60662759  1.86563634
  -0.60591545]
 [-0.93339552 -0.23817242 -0.58444712 ..., -0.23626019 -0.23742748
  -0.93245275]]


In [21]:
def print_recommendations(W=W, Q=Q, Q_hat=Q_hat, df=rp):
    music_ids = np.argmax(Q_hat, axis=1)    # 랭크에서 가장 큰 값.
    rank_arr = np.sort(Q_hat)    # 각 행별 rank
    t_Q = np.copy(Q_hat)
    o_Q = np.copy(Q)
    
    for jj, music_id in zip(range(m), music_ids): # jj: row index
        print('Cluster, Tsection >> {} effected: \n'.format(df.index[jj]))
        print('Recommended MUSIC ==========> \n')
        for ii in range(30): # ii: max column index
            index_max = np.argmax(t_Q, axis=1)[jj]
            o_index_max = np.argmax(o_Q, axis=1)[jj]
            print ("ID: " + str(index_max) + ", Score: " + str(t_Q[jj][index_max]) + ", " + df.columns[index_max])
            #print Y.T[index_max]
            # print ("Original Score: " + str(o_Q[jj][o_index_max]) + ", " + music_titles[o_index_max].decode('cp949') )
            t_Q[jj][index_max] = -sys.maxint
            # o_Q[jj][o_index_max] = -sys.maxint
        print('\n' + 100 *  '-' + '\n')

In [22]:
def rating_recommendations(W=W, Q=Q, Q_hat=Q_hat, rp=rp):
    predict_rating = DataFrame({})
    music_ids = np.argmax(Q_hat, axis=1)    # 랭크에서 가장 큰 값.
    rank_arr = np.sort(Q_hat)    # 각 행별 rank
    t_Q = np.copy(Q_hat)
    o_Q = np.copy(Q)
    df = rp.copy()
    df = df.append(mf)
    f_Q = df.get_values()[-1]
    alpha = 0.7
    print f_Q
    
    for jj in range(m): # jj: row index
        predict_temp = DataFrame({})
        music_lst = []
        rating_lst = []
        score_lst = []
        cluster_lst = []
        season_lst = []
        tsect_lst = []
        hsect_lst = []
        t_Q[jj] = alpha * t_Q[jj] + (1-alpha) * f_Q
        for ii in range(100): # ii: max column index
            index_max = np.argmax(t_Q, axis=1)[jj]
            o_index_max = np.argmax(o_Q, axis=1)[jj]
            music_lst.append(df.columns[index_max])
            rating_lst.append(ii+1)
            score_lst.append(t_Q[jj][index_max])
            cluster_lst.append(df.index[jj][0])
            season_lst.append(df.index[jj][1])
#             tsect_lst.append(df.index[jj][1])
#             hsect_lst.append(df.index[jj][2])
            #print Y.T[index_max]
            # print ("Original Score: " + str(o_Q[jj][o_index_max]) + ", " + music_titles[o_index_max].decode('cp949') )
            t_Q[jj][index_max] = -sys.maxint # replace 방지
            # o_Q[jj][o_index_max] = -sys.maxint
        predict_temp['Music'] = music_lst
        predict_temp['Rank'] = rating_lst
        predict_temp['Score'] = score_lst
        # train variable
        predict_temp['Cluster'] = cluster_lst
        predict_temp['Season'] = season_lst
#         predict_temp['Tsection'] = tsect_lst
#         predict_temp['Hsection'] = hsect_lst
        predict_temp['Score']
        predict_rating = predict_rating.append(predict_temp, ignore_index=True)
    
    return predict_rating

In [23]:
rr = rating_recommendations()
rr

[-0.53771267  0.73494823  0.35940895 ..., -0.68375573 -0.60030255
  0.00473296]


,Music,Rank,Score,Cluster,Season
0,그런 사람 또 없습니다%by%이승철,1,6.392595,1,fall
1,소주 한 잔%by%임창정,2,4.930769,1,fall
2,Never Ending Story%by%부활,3,4.722790,1,fall
3,여전히 아름다운지 (feat. 김연우)%by%토이(Toy),4,4.241844,1,fall
4,거리에서%by%성시경,5,4.114890,1,fall
5,끝사랑%by%김범수,6,3.943643,1,fall
6,귀로%by%나얼(Na-Ul),7,3.839792,1,fall
7,너를 위해%by%임재범,8,3.784732,1,fall
8,바람이 분다%by%이소라,9,3.761207,1,fall
9,다시 사랑한다 말할까%by%김동률,10,3.709844,1,fall


In [24]:
target_info = []
target_chart = []
target_date = date(2012, 1, 1)
for i in range(1461):
    target_date_int = target_date.year * 10000 + target_date.month * 100 + target_date.day    
    target_info.append(weathers[weathers.Date == target_date_int])
    target_chart.append(mania_chart[mania_chart.Date == target_date_int])
    target_date = target_date + td

In [25]:
target_info[44]

,Date,meanT,maxT,minT,dewPoint,avgH,maxH,minH,pressure,avgW,...,H6,H7,H8,H9,W0,W1,Tsection,Hsection,Wsection,wcnt
44,20120214,3,8,-2,-8,52,69,27,1021.29,7,...,0,0,0,0,1,0,1,1,1,1


In [26]:
jc = {}
JacardDF = DataFrame(jc)
JC_fm_lst = []
JC_target_lst = []
intersect_lst = []
origin_correct_lst = []
correct_lst = []
cur_date_lst = []

cur_date = date(2012, 1, 1)
fm = freq_songs[:100]['Title']

for j in range(1461):
    try:
        # Test Set Condition
        target_data = weathers_test[weathers_test['Date'] == target_info[j]['Date'].get_values()[0]]
        if not target_data.shape[0]:
            cur_date = cur_date + td
            continue

        target_score_lst = []
        name_lst = []
        id_lst = []
        temp = {}
        lrdf = DataFrame(temp)

        target_cluster = target_info[j]['cluster'].get_values()[0]
        target_season = target_info[j]['Season'].get_values()[0]
#         target_tsect = target_info[j]['Tsection'].get_values()[0]//2
#         target_hsect = target_info[j]['Hsection'].get_values()[0]//2

        cluster_rec = rr[rr['Cluster'] == target_cluster]
        target_recommendations = cluster_rec[cluster_rec['Season'] == target_season]
#         tsect_rec = cluster_rec[cluster_rec['Tsection'] == target_tsect]
#         target_recommendations = tsect_rec[tsect_rec['Hsection'] == target_hsect]

        #target_chart[j]['name'] = target_chart[j].Title + '%by%' + target_chart[j].Artist

        tc = target_chart[j]['Title']
        tc.columns = ['Title']
        tr = target_recommendations['Music']

        t_intersect = pd.Series(list(set(tr) & set(tc)))
        t_union = pd.Series(list(set(tr) | set(tc)))
        JC_target = t_intersect.count() / float(t_union.count())

        fm_intersect = pd.Series(list(set(fm) & set(tc)))
        fm_union = pd.Series(list(set(fm) | set(tc)))
        JC_fm = fm_intersect.count() / float(fm_union.count())

        last_intersect = pd.Series(list(set(t_intersect) & set(fm_intersect)))
        last_union = pd.Series(list(set(fm_intersect) | set(t_intersect)))
        JC_last = last_intersect.count() / float(last_union.count())
        print str(cur_date) + '==============='
        print JC_fm, JC_target
        #print 'Original Rank ============'
        #print fm_intersect
        #print 'Regression Rank ============'
#         print tr
        print 'cross ========'
        #print JC_last
        print t_intersect.count()
        #print t_intersect
        #print last_intersect
        JC_fm_lst.append(JC_fm)
        JC_target_lst.append(JC_target)
        intersect_lst.append(last_intersect.count())
        origin_correct_lst.append(fm_intersect.count())
        correct_lst.append(t_intersect.count())
        cur_date_lst.append(str(cur_date))

        cur_date = cur_date + td
    except IndexError as e:
        print "There is not " + str(cur_date)
        cur_date = cur_date + td
        continue

JacardDF['Original'] = JC_fm_lst
JacardDF['Logit'] = JC_target_lst
JacardDF['O_Correct'] = origin_correct_lst
JacardDF['L_Correct'] = correct_lst
JacardDF['noIntersect'] = intersect_lst
JacardDF['Date'] = cur_date_lst

2012-01-02===============
0.0810810810811 0.063829787234
cross ========
12
2012-01-03===============
0.0752688172043 0.0582010582011
cross ========
11
2012-01-05===============
0.0695187165775 0.0752688172043
cross ========
14
2012-01-06===============
0.0471204188482 0.0582010582011
cross ========
11
2012-01-10===============
0.0362694300518 0.0526315789474
cross ========
10
2012-01-15===============
0.0309278350515 0.0471204188482
cross ========
9
2012-01-19===============
0.025641025641 0.0152284263959
cross ========
3
2012-01-20===============
0.0204081632653 0.0152284263959
cross ========
3
2012-01-28===============
0.0152284263959 0.025641025641
cross ========
5
2012-01-30===============
0.0152284263959 0.0204081632653
cross ========
4
2012-02-01===============
0.0204081632653 0.0309278350515
cross ========
6
2012-02-04===============
0.0152284263959 0.063829787234
cross ========
12
2012-02-09===============
0.0869565217391 0.0928961748634
cross ========
17
2012-02-10============

In [27]:
JacardDF.to_csv('ALS/Jacard_Cluster_Tsection_Hsection_hybrid_over30_test02_L01F100.csv', encoding='utf-8-sig')
prevDF = JacardDF
JacardDF.describe()

,Original,Logit,O_Correct,L_Correct,noIntersect
count,292.000000,292.000000,292.000000,292.000000,292.000000
mean,0.072067,0.084185,13.315068,15.297945,10.119863
std,0.026600,0.037273,4.593067,6.193633,4.049014
min,0.015228,0.015228,3.000000,3.000000,1.000000
25%,0.052632,0.058201,10.000000,11.000000,7.000000
50%,0.069892,0.081081,13.000000,15.000000,10.000000
75%,0.086957,0.104972,16.000000,19.000000,13.000000
max,0.150289,0.206061,26.000000,34.000000,22.000000


In [28]:
prevDF.describe()

,Original,Logit,O_Correct,L_Correct,noIntersect
count,292.000000,292.000000,292.000000,292.000000,292.000000
mean,0.072067,0.084185,13.315068,15.297945,10.119863
std,0.026600,0.037273,4.593067,6.193633,4.049014
min,0.015228,0.015228,3.000000,3.000000,1.000000
25%,0.052632,0.058201,10.000000,11.000000,7.000000
50%,0.069892,0.081081,13.000000,15.000000,10.000000
75%,0.086957,0.104972,16.000000,19.000000,13.000000
max,0.150289,0.206061,26.000000,34.000000,22.000000


In [85]:
print_recommendations(Q_hat=weighted_Q_hat)

Cluster (1, 1.0) effected: 

Recommended MUSIC ==========> 

ID: 338, Score: 3.99044490362, 벌써 일년%by%브라운 아이즈(Brown Eyes)
ID: 531, Score: 3.81757365777, 잘못된 만남%by%김건모
ID: 191, Score: 3.15486441378, 그런 사람 또 없습니다%by%이승철
ID: 275, Score: 3.14800491097, 다시 사랑한다 말할까%by%김동률
ID: 251, Score: 3.14333222572, 너랑 나%by%아이유(IU)
ID: 430, Score: 2.97429734462, 술 한잔 해요%by%지아(Zia)
ID: 144, Score: 2.80277825903, 겨울 이야기 (Radio ver.)%by%DJ DOC
ID: 253, Score: 2.80188328139, 너를 위해%by%임재범
ID: 345, Score: 2.64410751038, 보고 싶다%by%김범수
ID: 224, Score: 2.63646642374, 나였으면%by%나윤권
ID: 172, Score: 2.63379507185, 그녀를 사랑해줘요%by%하동균
ID: 368, Score: 2.47493581845, 비와 당신 (영화 '라디오 스타 '삽입곡)%by%럼블 피쉬(Rumble Fish)
ID: 482, Score: 2.46929482682, 여전히 아름다운지 (feat. 김연우)%by%토이(Toy)
ID: 203, Score: 2.46766085254, 기억을 걷는 시간%by%넬(Nell)
ID: 83, Score: 2.46741190497, Officially Missing You%by%긱스(Geeks)
ID: 379, Score: 2.30216501097, 사랑.. 그 놈%by%바비 킴(Bobby Kim)
ID: 321, Score: 2.30014829389, 미워요%by%정인
ID: 249, Score: 2.29815956053, 내입술...

In [102]:
print X[1] # Hot, Rain
print X[2] # Hot, Rain, Thunderstorm
print X[0] # Cold
print X[6] # Cold, Snow

[-3.61602958  0.70141116  0.56606788  0.54810534 -0.6369384  -1.80901893
 -0.50914743  2.73693037  1.16210215  1.63650659  0.43395775  1.14627584
 -2.30019893  1.47198469  0.29615213  1.97226565 -1.86482332 -0.8443866
 -2.27534499  0.7655833 ]
[-2.23870016 -1.25311333 -1.15846799 -0.33771943  0.2823995  -0.6200145
  2.18905037  1.60589067  1.88658798  0.95731231  0.89195556  1.52713515
 -1.38064108  0.66547329 -1.72755936  0.20790518 -0.7782951  -0.24611698
 -2.19009422  1.41011057]
[-1.89739569  1.61635113 -0.90983203  1.10291784 -0.79726299 -1.09755059
 -0.06344435 -0.85893247 -1.11197204  1.25358348 -0.96085335  2.13263043
  0.19890618  1.06573885 -0.81200548  1.30682364  2.05890064 -0.70197284
 -2.1843752   0.39208016]
[-0.47587082  1.84709372 -1.26136304 -0.62338181  2.46306803 -1.26116306
  0.20445595 -0.76007398 -0.98075052 -0.41566471  0.53505285  0.08557788
 -0.85368982  1.85624906 -1.02017151  0.34833187  2.70685998 -0.47500836
 -0.92392088 -1.05009674]


In [66]:
print Y.T[505] # 우산 - 윤하
print Y.T[610] # 해변의 여인 - 쿨
print Y.T[598] # 팥빙수 - 윤종신
print Y.T[3] # 12월 32일 - 별

[  4.68073580e-02  -3.94146935e-03   1.95774787e-02  -1.72854049e-02
  -3.41463911e-02   1.86071614e-02   3.99295493e-02   1.22557856e-02
  -4.24131548e-02  -5.04569613e-03   2.07632808e-02   2.53139082e-02
   3.55541061e-03   5.07769995e-02  -2.35749041e-02  -4.99544057e-02
  -4.96939672e-02   5.60462982e-02  -4.60251010e-02   8.98976346e-05
  -3.51654837e-02   2.46279970e-03  -1.26173664e-02  -5.96482645e-03
  -1.82942908e-02  -9.03075683e-02  -4.93089162e-02   2.10375063e-02
  -4.18964956e-02   1.07087212e-02   9.55097045e-03   5.19865256e-02
  -4.02623300e-02   1.63550822e-02   5.20113302e-02   3.93718895e-02
  -3.97204041e-02   1.44768351e-02   5.34675520e-02   3.83774661e-02
  -4.54493990e-02  -5.11686359e-03   7.55356788e-02   4.30638057e-02
  -1.63190991e-02   4.23118102e-02  -2.20982189e-02  -4.62352274e-02
  -4.88624989e-03   2.24298988e-02   3.04257253e-03   2.95953274e-02
  -2.44355984e-03   3.12675154e-02   3.70085406e-02   4.53547620e-02
  -3.14227734e-02  -4.85627705e-02

In [67]:
print weighted_Q_hat[1][505]
print weighted_Q_hat[1][598]
print weighted_Q_hat[1][610]
print weighted_Q_hat[1][619]
print weighted_Q_hat[6][3]
print Q[6][3]

3.4365065382
-0.0079874554418
2.90679923164
-1.33354202524
2.89922105643
2.89952325319


In [48]:
def get_error(Q, P):
    return np.sum((Q - P) ** 2)

In [59]:
origin_ = Q[W == 0]
predict_ = weighted_Q_hat[W == 0]
error_lst = Q[W==0] - weighted_Q_hat[W==0]
print sum(abs(error_lst))
Q[W==0]
# print get_error(origin_, predict_)

73.9435123401


array([ 0.68773148,  2.07646258, -1.04818239,  3.81237644,  0.68773148,
       -1.04818239,  0.34054871, -0.70099961, -0.70099961,  0.34054871,
       -1.04818239, -0.00663407, -0.00663407, -0.00663407,  0.70020194,
        3.03917436,  0.70020194, -0.85911301, -0.07945554, -0.85911301,
       -0.85911301,  0.70020194, -0.85911301,  0.61804172,  0.61804172,
        0.61804172,  0.61804172, -0.81947754, -0.10071791, -0.10071791,
        2.05556099,  2.77432062, -0.81947754, -0.81947754, -0.81947754,
       -0.10071791, -0.81947754, -0.81947754, -0.69801563, -0.69801563,
        1.36482586,  2.3962466 ,  0.33340511,  0.33340511,  0.33340511,
       -0.69801563,  1.36482586, -0.69801563, -0.69801563,  0.33340511,
        0.33340511, -0.69801563,  0.33340511, -0.69801563, -0.69801563,
       -0.69801563, -0.69883795, -0.69883795, -0.69883795, -0.69883795,
       -0.69883795,  1.30513571, -0.69883795, -0.69883795, -0.69883795,
        1.30513571,  0.30314888,  3.30910936, -0.46894879, -0.46

In [68]:
np.dot(X[1], Y.T[505])

3.4365065382044437

In [84]:
rp.describe()

Title,...사랑했잖아...%by%린(LYn),...안 되나요... (부제: 화양연화)%by%휘성,"1,2,3,4 (원,투,쓰리,포)%by%이하이",12월 32일%by%별,1994년 어느 늦은 밤%by%장혜진,1월부터 6월까지 (feat. 윤종신)%by%공일오비(015B),3!4!%by%룰라(Roo'Ra),365일%by%알리(Ali),7년간의 사랑%by%규현(KYUHYUN),"? (물음표) (feat. 최자 Of 다이나믹듀오, Zion.T)%by%프라이머리(Primary)",...,홀로 (feat. 김나영)%by%정키(Jung Key),홀로 크리스마스 (With 나비)%by%마이티 마우스(Mighty Mouth),홍대앞에 눈이 내리면%by%윤건,화분 (알렉스 ver.) ('우리 결혼했어요' 삽입곡)%by%알렉스(Alex) [클래지콰이],화이트 (White)%by%핑클(Fin.K.L),화장을 고치고%by%왁스(Wax),환상%by%박지윤,회상 (December)%by%터보(Turbo),후회한다%by%포맨,흰눈%by%이루(Eru)
count,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
mean,0.105915,-0.291801,-0.444511,0.069938,-0.467350,-0.271488,-0.049659,0.750857,-0.533270,0.362012,...,0.498464,-0.483456,-0.201943,-0.497651,-0.525623,0.481286,-0.613881,0.480612,-0.355028,-0.213020
std,0.469892,0.388608,0.277809,1.480254,0.708902,0.325338,1.398220,0.818143,0.511602,0.495326,...,0.753061,1.458095,1.047043,0.608047,0.986316,0.660184,0.490025,1.083547,0.964281,1.177025
min,-0.462657,-1.046652,-0.803488,-1.467244,-0.935996,-0.616293,-1.551500,-0.339690,-1.320754,-0.339690,...,-0.498447,-1.482401,-1.328870,-1.046652,-1.482401,-0.498447,-1.085968,-0.844563,-1.398057,-1.467244
25%,-0.184027,-0.454954,-0.616041,-0.974530,-0.857863,-0.516601,-0.943086,0.033177,-0.867421,-0.009911,...,0.003886,-1.366951,-0.963230,-0.891874,-1.366951,0.120375,-0.954935,-0.336004,-1.131567,-0.967107
50%,0.079672,-0.181415,-0.485431,-0.149189,-0.788905,-0.405169,-0.710988,0.750016,-0.581879,0.399016,...,0.490817,-1.081611,-0.467813,-0.729380,-0.726446,0.456223,-0.774329,0.296897,-0.453238,-0.588069
75%,0.266196,-0.073449,-0.374072,0.655372,-0.493218,0.057486,1.131071,1.342547,-0.097106,0.566728,...,0.712452,-0.406366,0.579682,-0.257144,0.196121,1.069531,-0.420000,1.421657,0.270778,0.356295
max,1.051624,0.164878,0.035292,2.899523,0.949296,0.164878,2.242576,2.047482,0.123055,1.263064,...,2.010251,2.668151,1.279917,0.662719,0.817172,1.298605,0.354427,1.901573,1.279917,2.205406


In [85]:
crp = rp.copy()
crp = crp.T
crp.describe()

cluster,1,2,3,4,5,6,7,8
count,6.280000e+02,6.280000e+02,6.280000e+02,6.280000e+02,6.280000e+02,6.280000e+02,6.280000e+02,6.280000e+02
mean,7.354344e-17,-6.647195e-17,-1.060723e-16,1.131437e-17,-9.334359e-17,-1.329439e-16,3.677172e-17,9.051500e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.623439e+00,-1.333520e+00,-1.320754e+00,-1.482401e+00,-1.500787e+00,-1.403911e+00,-1.265180e+00,-1.467244e+00
25%,-6.882319e-01,-6.709801e-01,-7.725494e-01,-6.979825e-01,-8.424676e-01,-6.090535e-01,-8.024348e-01,-7.061895e-01
50%,-1.127197e-01,-1.409480e-01,-2.243449e-01,-2.273315e-01,-1.841478e-01,-1.321388e-01,-1.083176e-01,-1.526956e-01
75%,5.347315e-01,3.890841e-01,5.979620e-01,4.786451e-01,4.741720e-01,3.447759e-01,5.857995e-01,5.391718e-01
max,4.347500e+00,3.701785e+00,4.435394e+00,3.694761e+00,4.643531e+00,4.319065e+00,5.444619e+00,5.105496e+00


# 함수 적용 테스팅

In [ ]:
def testing():
    jc = {}
    JaccardDF = DataFrame(jc)
    JC_fm_lst = []
    JC_target_lst = []
    intersect_lst = []
    cur_date_lst = []
    origin_correct_lst = []
    correct_lst = []
    JC_OnL_lst = []

    cur_date = date(2012, 1, 1)
    fm = freq_songs[:100]['Title']

    model_lst = []

    for i in range(628):
        model = LogisticRegression() 
        model.fit(X_train, y_train[y_train.columns[i]])
    #     test_score_lst.append(model_lst[i-1].predict_proba(X_test)[:,1])
        model_lst.append(model)

    for j in range(1461):
        try:
            target_data = ainfo_test[ainfo_test['Date'] == target_info[j]['Date'].get_values()[0]]
            if not target_data.shape[0]:
                cur_date = cur_date + td
                continue

            target_score_lst = []
            name_lst = []
            id_lst = []
            temp = {}
            lrdf = DataFrame(temp)

            tarX = dmatrix(features, target_info[j], return_type="dataframe")

            for i in range(628):
        #             train_score_lst.append(model2.predict_proba(X_train)[:,1])
        #             test_score_lst.append(model_lst[i-1].predict_proba(X_test)[:,1])
                target_score_lst.append(model_lst[i].predict_proba(tarX)[:,1][0])
                id_lst.append(i)

                # generate evaluation metrics
                title_name = freq_songs.loc[i].get_value('Title')
                name_lst.append(title_name)

            lrdf['Score(target)'] = target_score_lst
            lrdf['Title'] = name_lst
            lrdf['ID'] = id_lst

            target_rating = lrdf.sort_values('Score(target)', ascending=False)[:100]

            target_chart[j]['name'] = target_chart[j].Title + '%by%' + target_chart[j].Artist

            t2 = target_chart[j]['name']
            t1 = target_rating['Title']
            t2.columns = ['Title']
            #t3 = t1.append(t2)
            #t3 = t1.intersection(t2)
            t_intersect = pd.Series(list(set(t1) & set(t2)))
            t_union = pd.Series(list(set(t1) | set(t2)))
            JC_target = t_intersect.count() / float(t_union.count())

            fm_intersect = pd.Series(list(set(fm) & set(t2)))
            fm_union = pd.Series(list(set(fm) | set(t2)))
            JC_fm = fm_intersect.count() / float(fm_union.count())

            last_intersect = pd.Series(list(set(t_intersect) & set(fm_intersect)))
            last_union = pd.Series(list(set(fm_intersect) | set(t_intersect)))
            JC_last = last_intersect.count() / float(last_union.count())
            print str(cur_date) + '==============='
            print 'Recomm Correct: ', t_intersect.count()
            print JC_fm, JC_target
            #print 'Original Rank ============'
            #print fm_intersect
            #print 'Regression Rank ============'
            #print t_intersect
            print 'cross ========'
            print JC_last
        #     print last_intersect
            JC_fm_lst.append(JC_fm)
            JC_target_lst.append(JC_target)
            intersect_lst.append(last_intersect.count())
            origin_correct_lst.append(fm_intersect.count())
            JC_OnL_lst.append(JC_last)
            correct_lst.append(t_intersect.count())
            cur_date_lst.append(str(cur_date))

            cur_date = cur_date + td
        except IndexError as e:
            print "There is not " + str(cur_date)
            cur_date = cur_date + td
            continue

    JaccardDF['JC_Original'] = JC_fm_lst
    JaccardDF['JC_Logit'] = JC_target_lst
    JaccardDF['O_Correct'] = origin_correct_lst
    JaccardDF['L_Correct'] = correct_lst
    JaccardDF['JC_OnL'] = JC_OnL_lst
    JaccardDF['noIntersect'] = intersect_lst
    JaccardDF['Date'] = cur_date_lst
    return JaccardDF